In [ ]:
import numpy as np

def calcular_cinematica_inversa(T_total, L1, L2, L3, L4):
    """
    Calcula la cinemática inversa para un brazo robótico de 4 grados de libertad (DOF).

    Parámetros:
        T_total: Matriz de transformación homogénea 4x4 para la pose del efector final (en milímetros).
        L1, L2, L3, L4: Longitudes de los eslabones en metros.

    Retorna:
        tuple: (theta1, theta2, theta3, theta4) en grados.
    """
    # Convertir T_total a float64 y luego a metros
    factor_escala = 0.001  # mm a m
    T_total_m = T_total.astype(np.float64)
    T_total_m[:3, 3] = T_total_m[:3, 3] * factor_escala

    # Extraer la posición de T_total
    Pc = T_total_m[:3, 3]

    # Paso 1: Calcular theta1 (rotación de la base)
    theta1 = np.arctan2(Pc[1], Pc[0])

    # Paso 2: Calcular la posición en el plano del brazo
    # Proyectar el punto en el plano del brazo después de la rotación de la base
    r = np.sqrt(Pc[0]**2 + Pc[1]**2)  # Distancia radial en el plano XY
    z = Pc[2]  # Altura

    # Ajustar la posición objetivo para considerar L4 (longitud del efector final)
    # Trabajamos hacia atrás desde el efector final hasta la posición de la muñeca
    R = T_total_m[:3, :3]  # Matriz de rotación
    vector_aproximacion = R[:, 2]  # Tercera columna de la matriz de rotación
    posicion_muñeca = Pc - L4 * vector_aproximacion

    # Recalcular r y z para la posición de la muñeca
    r_muñeca = np.sqrt(posicion_muñeca[0]**2 + posicion_muñeca[1]**2)
    z_muñeca = posicion_muñeca[2]

    # Paso 3: Calcular theta2 y theta3 usando el enfoque geométrico
    try:
        # Distancia desde el hombro hasta la muñeca
        c = np.sqrt(r_muñeca**2 + (z_muñeca - L1)**2)

        # Verificar si el punto es alcanzable
        if c > (L2 + L3) or c < abs(L2 - L3):
            raise ValueError(f"El punto ({r_muñeca:.3f}, {z_muñeca:.3f}) está fuera del espacio de trabajo alcanzable.")

        # Calcular theta2
        cos_theta2 = (L2**2 + c**2 - L3**2) / (2 * L2 * c)
        alpha = np.arctan2(z_muñeca - L1, r_muñeca)
        theta2 = np.arccos(cos_theta2) + alpha

        # Calcular theta3
        cos_theta3 = (L2**2 + L3**2 - c**2) / (2 * L2 * L3)
        theta3 = -(np.pi - np.arccos(cos_theta3))

        # Calcular theta4 (ángulo de la muñeca)
        # La orientación total es la suma de todos los ángulos articulares
        theta4 = -theta2 - theta3

        # Convertir a grados
        theta1_deg = np.degrees(theta1)
        theta2_deg = np.degrees(theta2)
        theta3_deg = np.degrees(theta3)
        theta4_deg = np.degrees(theta4)

        return theta1_deg, theta2_deg, theta3_deg, theta4_deg

    except ValueError as e:
        raise ValueError(f"Error en la cinemática inversa: {str(e)}")

# Prueba de la función con matrices de transformación
if __name__ == "__main__":
    # Matriz de transformación de prueba (en milímetros)
    T_total = np.array([[0, -1,  0, 79],
                        [0,  0,  1,  0],
                        [-1,  0,  0, 35],
                        [0,  0,  0,  1]])

    # Longitudes de los eslabones en metros
    L1 = 0.35  # m
    L2 = 0.30  # m
    L3 = 0.30  # m
    L4 = 0.19  # m

    try:
        theta1, theta2, theta3, theta4 = calcular_cinematica_inversa(T_total, L1, L2, L3, L4)
        print(f"Theta 1: {theta1:.2f}°")
        print(f"Theta 2: {theta2:.2f}°")
        print(f"Theta 3: {theta3:.2f}°")
        print(f"Theta 4: {theta4:.2f}°")

        # Información adicional para depuración
        print("\nInformación de depuración:")
        print(f"Posición objetivo (m): ({T_total[0,3]*0.001:.3f}, {T_total[1,3]*0.001:.3f}, {T_total[2,3]*0.001:.3f})")
        r = np.sqrt((T_total[0,3]*0.001)**2 + (T_total[1,3]*0.001)**2)
        print(f"Distancia radial (m): {r:.3f}")
        print(f"Alcance máximo (m): {L1 + L2 + L3 + L4:.3f}")

    except ValueError as e:
        print(f"Error: {e}")